Code Part for plotting the respective Graphs. 

Importing the required libraries    

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.utils import resample
from scipy.interpolate import interp1d  # For smooth interpolation
from pathlib import Path  # For file path management


Load the data 